# RADMC-3D Volume Rendering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.animation import FuncAnimation
from tqdm.auto import tqdm
import seaborn as sns
import astropy.constants as c
from IPython.display import Video

import radmc3d_volume_rendering as rvr

au = c.au.cgs.value

## Setup

We need to set the path to the radmc3d source folder `src` when initializing. It will then compile the 'normal' radmc3d together with the userdefined function.

In [ ]:
ren = rvr.Renderer(src_dir='~/CODES/radmc3d-2.0/src/')

Next, we need some data: grid and densities. You can assign them directly:


```python
ren.t_i = theta_ii
ren.p_i = phi_i
ren.r_i = r_i
ren.rho = rho
```

where $\theta$ is defined to start from 0 at the pole and the order of indices of the density is: $\rho(r, \theta, \phi)$.

If those variables are stored under the same names (`r_i, t_i, p_i, rho`) we can load that directly (or a dictionary with the same keywords):

In [ ]:
ren.read_data('fargo3d_data.npz')

And to write out the density for radmc3d, we do:

In [ ]:
ren.write_input()

We write out the desired parameters of a gaussian transfer function:

**WARNING:** right now this is only read-in once, so this needs to be fixed in the image calculation, perhaps defining this as `myaction`? Right now, if you change those parameters, you need to restart the child process with `ren._stop_radmc3d_child()`

In [ ]:
ren.write_transfer_options(mean=0.5 * ren.rho.max(), sigma=0.1 * ren.rho.max())
ren._stop_radmc3d_child()

## Rendering

Rendering should now work by directly talking to the child process:

In [ ]:
cmd = 'image lambda 5 sizeradian 1.0 projection 1 posang 0 locobsau 1 -0.25 0.25 pointau 0.98 0.19 0 nofluxcons npix 1000'

In [ ]:
%time im = ren.make_image(cmd)

but `ren.callit` and `ren.plotit` should still work for calls to radmc3d without using the child-process method.

## Plot the image

In [ ]:
vmax = im.im.max()

f = plt.figure(figsize=(5, 5), dpi=150)
ax = f.add_axes([0, 0, 1, 1])
ax.pcolormesh(im.xi, im.yi, im.im.T, norm=LogNorm(vmin=1e-5*vmax, vmax=1e-1*vmax, clip=True), cmap='rocket')
ax.set_aspect(1)
ax.axis('off')
f.savefig('example.jpg')

Sanity check: vertical sum of density to see if the pattern matches

In [ ]:
Ri, PHi = np.meshgrid(ren.r_i / c.au.cgs.value, ren.p_i, indexing='ij')
_Xi = Ri * np.cos(PHi)
_Yi = Ri * np.sin(PHi)

f, ax = plt.subplots()
ax.pcolormesh(_Xi, _Yi, (ren.rho * 0.5 * (ren.r_i[1:]+ ren.r_i[:-1])[:, None, None] * np.diff(ren.t_i)[None, :, None]).sum(1))
ax.set_aspect('equal')

## Movie

Basic pointing and image size

In [ ]:
pnt = np.array([0.92, 0.38, 0]) # where the camera points, needs to be printed in cm later
obs = np.array([1, -0.25, 0.25]) # observer position, needs to be changed as well
hsx, hsy = [1.0, 0.5625] # x and y extent of the image in radian
pa = 0  # position angle of camera

Define the observer position by panning around the point, starting at that initial position

In [ ]:
# get radius of the motion
vec = pnt[:2] - obs[:2]
r_pan = np.hypot(*vec)

# define a rotation angle array
phi0 = np.arctan2(*vec[::-1])
phi_pan = phi0 + np.linspace(0, 2 * np.pi, 5 * 24)

# do a 360 degree pan around that point
obsx = pnt[0] + r_pan * np.cos(phi_pan)
obsy = pnt[1] + r_pan * np.sin(phi_pan)

Plot the camera path

In [ ]:
_R,_P = np.meshgrid(ren.r_i / au, ren.p_i, indexing='ij')
_X = _R * np.cos(_P)
_Y = _R * np.sin(_P)

f, ax = plt.subplots(dpi=100)
ax.plot(0,0, 'k+')
s = 10
ax.plot(_X[::s, ::s], _Y[::s, ::s], 'k-')
ax.plot(_X[::s, ::s].T, _Y[::s, ::s].T, 'k-');

ax.plot(*obs[:2],'rv')
ax.plot(*pnt[:2],'bo')

ax.plot([obs[0], pnt[0]], [obs[1], pnt[1]] ,'b--')

ax.plot(obsx, obsy)
ax.set_aspect(1)

Write the movie input file

In [ ]:
with open(ren.path / 'movie.inp', 'w') as fid:
    # ifformat=1 means local observer mode
    # iformat=-1 means observer at infinity
    fid.write('-1\n')
    fid.write(f'{len(obsx):d}\n')
    for _x, _y in zip(obsx, obsy):
        _arr = [*pnt * au, hsx, hsy, pa, _x * au, _y * au, obs[-1] * au]
        fid.write(' '.join([f'{_v:.8g}' for _v in _arr]) + '\n')

Call the movie command (takes a bit over 3 minutes for me)

In [ ]:
%%time
N = 512
ren.callit(command=f'movie lambda 5 nofluxcons npixx {N} npixy {int(N * hsy / hsx)}')

Create the figure and the update function to make an animation out of it

In [ ]:
%%capture
im = rvr.radmc3d_volume_rendering.read_image(filename=str(ren.path / 'image_0001.out'))
vmax = im.image.max()

fig = plt.figure(figsize=(5, 5 / np.divide(*im.image.shape)), dpi=150)
ax = fig.add_axes([0, 0, 1, 1])
pcm = ax.pcolormesh(im.x, im.y, im.image[::-1, :].T,
              norm=LogNorm(vmin=1e-5*vmax, vmax=1e-1*vmax, clip=True),
              cmap='rocket', shading='auto')
ax.set_aspect(1)
ax.axis('off');

def update(i):
    im = rvr.radmc3d_volume_rendering.read_image(filename=str(ren.path / f'image_{i:04d}.out'))
    pcm.set_array(im.image[::-1, :].T.ravel())

Render the animation

In [ ]:
ani = FuncAnimation(fig, update, frames=tqdm(np.arange(1, len(obsx) + 1)))
ani.save('pan.mp4', fps=24, extra_args=['-vcodec', 'libx264'])

Display it 

In [ ]:
Video('pan.mp4', html_attributes='controls autoplay', width=500)

## Other output

We can also make RADMC-3D write the vtk_grid and data

In [ ]:
ren.callit(command='vtk_grid')

In [ ]:
ren.callit(command='vtk_gas_density')